In [3]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
dotenv_path = '../.env'
load_dotenv(dotenv_path)

# Get database credentials from environment variables
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Create the database connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

print("Connection to database established successfully.")

# --- Test Query ---
# Let's run a simple query to fetch the first 5 customers to verify the connection
query = "SELECT * FROM olist_customers LIMIT 5;"
df_test = pd.read_sql(query, engine)

df_test.head()

Connection to database established successfully.


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


RFM Analysis Explained: 
RFM is a powerful marketing analysis technique used to segment customers based on their behavior. It stands for:

Recency (R): How recently did the customer make a purchase?

Frequency (F): How often do they make purchases?

Monetary (M): How much money do they spend?


This query uses Common Table Expressions (CTEs) to build the logic step-by-step, which is a clean and efficient way to write complex SQL.

In [4]:
# This query joins customers, orders, and payments to calculate RFM metrics.
# We use Common Table Expressions (CTEs) to build the logic step-by-step.

rfm_query = """
WITH
  -- First, get the most recent purchase date in the entire dataset to calculate recency against a fixed point in time.
  snapshot_date AS (
    SELECT
      MAX(order_purchase_timestamp) AS max_purchase_date
    FROM
      olist_orders
  ),
  
  -- Next, join the necessary tables and filter for completed orders.
  customer_orders AS (
    SELECT
      c.customer_unique_id,
      o.order_id,
      o.order_purchase_timestamp,
      p.payment_value
    FROM
      olist_customers AS c
      JOIN olist_orders AS o ON c.customer_id = o.customer_id
      JOIN olist_order_payments AS p ON o.order_id = p.order_id
    WHERE
      o.order_status = 'delivered' AND o.order_purchase_timestamp IS NOT NULL
  )
  
-- Now, calculate the R, F, and M values for each unique customer.
SELECT
  co.customer_unique_id,
  -- Recency: Days since the customer's last purchase from the snapshot date.
  DATE_PART('day', (SELECT max_purchase_date FROM snapshot_date) - MAX(co.order_purchase_timestamp)) AS recency,
  -- Frequency: Total number of distinct orders.
  COUNT(DISTINCT co.order_id) AS frequency,
  -- Monetary: Total sum of payments.
  SUM(co.payment_value) AS monetary
FROM
  customer_orders AS co
GROUP BY
  co.customer_unique_id
ORDER BY
    monetary DESC;
"""

# Execute the query and load the results into a new DataFrame
df_rfm = pd.read_sql(rfm_query, engine)

print("RFM calculation complete.")
df_rfm.head()

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: text - text
LINE 30: ...y', (SELECT max_purchase_date FROM snapshot_date) - MAX(co.o...
                                                              ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 
WITH
  -- First, get the most recent purchase date in the entire dataset to calculate recency against a fixed point in time.
  snapshot_date AS (
    SELECT
      MAX(order_purchase_timestamp) AS max_purchase_date
    FROM
      olist_orders
  ),

  -- Next, join the necessary tables and filter for completed orders.
  customer_orders AS (
    SELECT
      c.customer_unique_id,
      o.order_id,
      o.order_purchase_timestamp,
      p.payment_value
    FROM
      olist_customers AS c
      JOIN olist_orders AS o ON c.customer_id = o.customer_id
      JOIN olist_order_payments AS p ON o.order_id = p.order_id
    WHERE
      o.order_status = 'delive